`establish_connection`

Establishes the connection to the database. Accepts a hash as input where the :adapter key must be specified with the name of a database adapter (in lower-case) example for regular databases (MySQL, PostgreSQL, etc

当调用一个不存在的方法时，Ruby会沿着祖先链向上查找这个方法。如果一直找到`BasicObject`还没有找到这个方法，则`Ruby`会回到祖先链底，调用`method_misssing`这个方法，这意味着幽灵方法的调用要慢于普通方法的调用，因为`Ruby`至少要完整的查找一遍祖先链

在大多数情况下，幽灵方法和普通方法的效率可以忽略不计。然而，在`Rails`中，属性方法的调用非常频繁。在`Rails 1`中，每一次对属性方法的调用都不得不遍历`ActiveRecord::Base`那么长的祖先链，因此效率非常的低。

In [3]:
class MyClass
  attr_accessor :my_attribute
  
  def set_attribute(n)
    my_attribute = n
  end
end

obj = MyClass.new
obj.set_attribute 10
p obj.my_attribute

nil


这个结果可能在你的意料之外。这是因为`self_attribute`的代码具有不确定性。`Ruby`不知道代码是想给局部变量赋值，还是调用名为`my_attribute=`的拟态方法。在没有确定答案的情况下，Ruby默认选择第一种方式，它定义了一个名为`my_attribute`的局部变量，该变量在复制完后就落在作用域之外了。

为了避免这个问题，给当前对象的属性赋值时，应该显性地使用`self`。

In [6]:
class MyClass
  def set_attribute(n)
    self.my_attribute = n
  end
end

obj.set_attribute 10
obj.my_attribute

10

自由方法（unbound method）更普通方法类似，不过它从最初定义它的类或者模块中脱离了。通过调用`Method#unbind`方法，可以把一个方法变成自由方法。你也可以直接调用`Module#instance_method`方法获得一个自由方法。下面是一个例子：

In [1]:
module MyModule
  def my_method
    42
  end
end

unbound = MyModule.instance_method(:my_method)
unbound.class

UnboundMethod

虽然不能调用`UnboundMethod`，但可以把它绑定到一个对象上，使之再次成为一个`Method`对象。具体的做法是使用`UnboundMehtod#bind`方法把`UnboundMethod`对象绑定到一个对象上。还可以把`UnboundMehtod`对象在传给`Module#define_method`方法，从而实现绑定。

In [2]:
String.send :define_method, :another_method, unbound
"abc".another_method

42